In [96]:
# Import libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
#import import_ipynb
import TP_WeatherAUS as TP
import importlib
importlib.reload(TP)
# Se ignoran los "FutureWarnings" molestos
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) 

In [51]:
# Read cvs file and generate copy
DATASET_PATH = 'weatherAUS.csv'
dataset = pd.read_csv(DATASET_PATH)
dataset_=dataset.copy()

In [52]:
# Observamos si hay fechas faltantes, ya que RainToday no coincide con RainTomorrow (deberían diferenciarse como máximo en 2 valores)

if 'Date' in dataset_.columns: dataset_ = dataset_.set_index('Date')
dataset_.index = pd.to_datetime(dataset_.index)
first_date = str(dataset_.index[0].year) + '-' + str(dataset_.index[0].month) + '-' + str(dataset_.index[0].day) 
last_date = str(dataset_.index[-1].year) + '-' + str(dataset_.index[-1].month) + '-' + str(dataset_.index[-1].day) 
print('el dataset va entre la fecha {} y la fecha {}'.format(first_date, last_date))

el dataset va entre la fecha 2008-12-1 y la fecha 2017-6-25


In [53]:
fechas_faltantes = pd.date_range(start=first_date, end=last_date).difference(dataset_.index)
print('entre las fechas 2008-12-01 y 2017-06-24 hay {} fechas faltantes en el dataset'.format(len(fechas_faltantes)))
print('\n las fechas son:', fechas_faltantes)

entre las fechas 2008-12-01 y 2017-06-24 hay 89 fechas faltantes en el dataset

 las fechas son: DatetimeIndex(['2011-04-01', '2011-04-02', '2011-04-03', '2011-04-04',
               '2011-04-05', '2011-04-06', '2011-04-07', '2011-04-08',
               '2011-04-09', '2011-04-10', '2011-04-11', '2011-04-12',
               '2011-04-13', '2011-04-14', '2011-04-15', '2011-04-16',
               '2011-04-17', '2011-04-18', '2011-04-19', '2011-04-20',
               '2011-04-21', '2011-04-22', '2011-04-23', '2011-04-24',
               '2011-04-25', '2011-04-26', '2011-04-27', '2011-04-28',
               '2011-04-29', '2011-04-30', '2012-12-01', '2012-12-02',
               '2012-12-03', '2012-12-04', '2012-12-05', '2012-12-06',
               '2012-12-07', '2012-12-08', '2012-12-09', '2012-12-10',
               '2012-12-11', '2012-12-12', '2012-12-13', '2012-12-14',
               '2012-12-15', '2012-12-16', '2012-12-17', '2012-12-18',
               '2012-12-19', '2012-12-20', '2012-12

In [54]:
# Contamos cuantas fechas faltantes hay en cada año, mes y día para ver algún patrón
fechas_faltantes_A = fechas_faltantes.year.value_counts()
fechas_faltantes_M = fechas_faltantes.month.value_counts()
fechas_faltantes_D = fechas_faltantes.day.value_counts()
print('cantidad de fechas faltantes en los años: \n', fechas_faltantes_A)
print('cantidad de fechas faltantes en los meses: \n',fechas_faltantes_M)
print('cantidad de fechas faltantes en los días: \n',fechas_faltantes_D)

cantidad de fechas faltantes en los años: 
 2012    31
2011    30
2013    28
dtype: int64
cantidad de fechas faltantes en los meses: 
 12    31
4     30
2     28
dtype: int64
cantidad de fechas faltantes en los días: 
 1     3
2     3
28    3
27    3
26    3
25    3
24    3
23    3
22    3
21    3
20    3
19    3
18    3
17    3
16    3
15    3
14    3
13    3
12    3
11    3
10    3
9     3
8     3
7     3
6     3
5     3
4     3
3     3
29    2
30    2
31    1
dtype: int64


## A PARTIR DE ACÁ SE HACEN LAS TRANSFORMACIONES PARA EL POSTERIOR ANÁLISIS DE LOS DATOS

In [55]:
# Se convierte la columna de fecha en dos columnas :año y día (entre 1 y 13).
dataset__=TP.date2columns(dataset)

In [56]:
# Nos quedamos con los años 2014, 2015 y 2016 que son los que tienen todas los días 
dataset__=dataset__[np.logical_or(np.logical_or(dataset__["Year"]==2014,dataset__["Year"]==2017), np.logical_or(dataset__["Year"]==2015, dataset__["Year"]==2016))]
dataset__=dataset__.reset_index(inplace=False).drop(columns="index")

In [57]:
dataset__['Year'].unique()

array([2014, 2015, 2016, 2017], dtype=uint16)

In [58]:
# Se eliminan las filas con datos faltantes de RainToday (mismas que faltantes en Rainfall)
# dataset__=dataset__[dataset__["RainToday"].notnull()]
# Se eliminan las filas con datos faltantes de RainTomorrow, que es nuestra variable de salida
# dataset__=dataset__[dataset__["RainTomorrow"].notnull()]

In [59]:
#Se verifican los null para cada columna 
#(en este caso particular busqueda ne null y de nan arrojan el mismo resultado)

for column in dataset__:
   tot=dataset__[column].isnull().sum()
   print(column,"porcentaje de Null",tot/len(dataset__[column])*100)

Year porcentaje de Null 0.0
MonthDay porcentaje de Null 0.0
Location porcentaje de Null 0.0
MinTemp porcentaje de Null 1.3910504275835514
MaxTemp porcentaje de Null 1.2915750798209442
Rainfall porcentaje de Null 2.7548253565870326
Evaporation porcentaje de Null 52.194073194602666
Sunshine porcentaje de Null 59.33864938148796
WindGustDir porcentaje de Null 6.18030709002519
WindGustSpeed porcentaje de Null 6.143404944887449
WindDir9am porcentaje de Null 6.892678935292891
WindDir3pm porcentaje de Null 3.754392157491938
WindSpeed9am porcentaje de Null 1.0380733871355912
WindSpeed3pm porcentaje de Null 3.0997801915702663
Humidity9am porcentaje de Null 1.9429781635567251
Humidity3pm porcentaje de Null 5.0844738235435685
Pressure9am porcentaje de Null 11.678726715548638
Pressure3pm porcentaje de Null 11.656264540247404
Cloud9am porcentaje de Null 41.606366422256805
Cloud3pm porcentaje de Null 45.64474465320006
Temp9am porcentaje de Null 1.2386285237537504
Temp3pm porcentaje de Null 4.36889309

### Analizamos las variables de temperatura, para hacer una imputación multivariada y eliminar las que tienen mayor cantidad de NANS y tienen alta correlación con sus pares

In [93]:
# Vemos que cantidad de valores nulos de manera simultánea tienen las variables de alta correlación
aux = dataset__[np.logical_and(dataset__['MinTemp'].isnull() , dataset__['Temp9am'].isnull())]
print('cantidad de filas con valores nulos simultaneos de MinTemp y Temp9am es {} '.format(len(aux)))

aux = dataset__[np.logical_and(dataset__['MaxTemp'].isnull() , dataset__['Temp3pm'].isnull())]
print('cantidad de filas con valores nulos simultaneos de MaxTemp y Temp3pm es {} '.format(len(aux)))

aux = dataset__[np.logical_and(dataset__['MaxTemp'].isnull() , True)]
print('cantidad de filas con valores de MaxTemp nulos es {} '.format(len(aux)))

cantidad de filas con valores nulos simultaneos de MinTemp y Temp9am es 732 
cantidad de filas con valores nulos simultaneos de MaxTemp y Temp3pm es 709 
cantidad de filas con valores de MaxTemp nulos es 805 


### Utilizamos el método multivariado de MICE para imputar valores faltantes mediante la función generada en TP_WeatherAUS-py:  "impute_column(df, col_to_predict, feature_columns)"

In [97]:
# Imputamos los valores faltantes a la variable MaxTemp, teniendo en cuenta las otras variables de temperatura

col_to_impute = 'MaxTemp'
print('Media de la variable previo a la imputación {}: {}'.format(col_to_impute, np.mean(dataset__[col_to_impute])))
feature_cols = ['MinTemp', 'Temp9am', 'Temp3pm']
datasetIMP = TP.impute_column(dataset__, col_to_impute, feature_cols)
print('Media de la variable posterior a la imputación {}: {}'.format(col_to_impute, np.mean(datasetIMP[col_to_impute])))

Media de la variable previo a la imputación MaxTemp: 23.77882546081077
61522
805


C:\Users\Aulamultimedia\anaconda3\envs\vision1\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\Aulamultimedia\MEGA\Especializacion en Inteligencia Artificial UBA\Materias\Bimestre 2\Analisis de datos\TP\analisis_datos_TP\TP_WeatherAUS.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3[col_to_predict].iloc[pred_rows_idx] = model.predict(X_pred.values)#.reshape(1,-1))


Media de la variable posterior a la imputación MaxTemp: 23.78027458129495


In [98]:
aux = datasetIMP[np.logical_and(datasetIMP['MaxTemp'].isnull() , True)]
print('cantidad de filas con valores de MaxTemp nulos es {} '.format(len(aux)))

cantidad de filas con valores de MaxTemp nulos es 0 
